Подготовим датасет к работе

In [216]:
import pandas as pd
import datetime

log = pd.read_csv("data/log.csv", header=None, names=['user_id','time', 'bet','win'])
display(log)

us = pd.read_csv('data/users.csv', encoding='koi8_r', sep='\t')
us.columns  = ['user_id', 'email', 'geo']
display(us)

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,NaN
996,#error,NaN,10054.0,29265.0
997,#error,NaN,10454.0,NaN
998,#error,NaN,1000.0,NaN


,user_id,email,geo
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,NaN
...,...,...,...
95,User_959,UpdatesCurious@yahoo.com,Тюмень
96,User_901,V2artierso@mail.ru,Арзангелтск
97,User_970,Vashoterlo@bk.ru,Воронеж
98,User_965,Visuareda@yahoo.com,Воронеж


Посчитайте количество пропусков в столбце time. 

Метод isna() есть не только у DataFrame, но и у Series. Это 

значит, что применять его можно не только ко всей таблице, но и к каждому столбцу отдельно.

In [217]:
log.time.isna().value_counts()

False    985
True      15
Name: time, dtype: int64

## Удаление пропусков

Пропуски можно удалять автоматически. Во многих случаях это правильно, так как данные с большим количеством пропусков часто не имеют смысла и не приносят никакой пользы.  

1. Удалять данные с пропусками можно с помощью метода dropna().
2. Параметр axis в методе dropna() говорит методу, по какой оси удалять значения.  

Если нужно удалить строки, в которых встречается пропуск (NaN), следует указать axis=0.  Зачем это делать? Например, у нас из 1000 примеров данных про пользователей пропуски есть в пяти. Разумно их удалить, так как их количество пренебрежимо мало.
Если нужно удалить столбцы, в которых встречается пропуск (NaN), нужно указывать axis=1. Зачем? Иногда в одном конкретном столбце пропусков настолько много, что с ними просто не хочется возиться - смысла в них все равно почти нет. 

Удалите все столбцы, где есть пропуски. Запишите в поле, сколько осталось столбцов в данных после этого.

In [218]:
display(log.columns)
log.dropna(axis=1).columns

Index(['user_id', 'time', 'bet', 'win'], dtype='object')

Index(['user_id'], dtype='object')

Удалите все строки, где есть пропуски. Запишите в поле, сколько осталось строк в данных после этого.

In [219]:
log.dropna(axis=0)

,user_id,time,bet,win
14,Запись пользователя № - user_917,[2019-01-02 8:57:36,145732.0,1987653.0
29,Запись пользователя № - user_942,[2019-01-04 13:59:42,1678321.0,9876543.0
151,Запись пользователя № - user_982,[2019-01-16 21:54:22,100.0,4749.0
189,Запись пользователя № - user_964,[2019-01-21 18:34:44,200.0,4667.0
205,Запись пользователя № - user_931,[2019-01-22 5:26:59,300.0,4319.0
...,...,...,...,...
967,Запись пользователя № - user_975,[2019-04-19 22:25:15,1000.0,6108.0
971,Запись пользователя № - user_912,[2019-04-20 10:35:49,10554.0,31799.0
972,Запись пользователя № - user_926,[2019-04-20 10:35:50,10354.0,30244.0
976,Запись пользователя № - user_970,[2019-04-20 10:35:54,10354.0,30691.0


Используйте оригинальный датасет log.csv.

С данными в столбцах bet и win мы разберемся позже: пропуски в этих столбцах требуют особого подхода.

А сейчас:

если есть пропуски в столбце user_id - удалите столбец user_id,

если есть пропуски в столбце time - удалите столбец time.

Запишите в поле ответа, количество оставшихся столбцов в данных, после этих действий.

Еще один интересный параметр - `subset`. Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск. То же самое работает и наоборот: нужно поменять axis на 1 и вместо названий столбцов передавать индексы строк.

In [220]:
log[['time', 'user_id']].dropna(axis=1)


,user_id
0,Запись пользователя № - user_919
1,Запись пользователя № - user_973
2,Запись пользователя № - user_903
3,Запись пользователя № - user_954
4,Запись пользователя № - user_954
...,...
995,Запись пользователя № - user_984
996,#error
997,#error
998,#error


Удалите дубли среди столбцов user_id и time. Запишите в поле ниже, сколько осталось строк после удаления дублей.

In [221]:
log.drop_duplicates(subset=['time', 'user_id'])

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN
...,...,...,...,...
991,Запись пользователя № - user_965,[2019-04-20 12:55:41,800.0,6927.0
992,Запись пользователя № - user_967,[2019-04-20 14:59:36,10154.0,NaN
993,Запись пользователя № - user_973,[2019-04-20 17:09:56,10254.0,NaN
994,Запись пользователя № - user_977,[2019-04-20 18:10:07,10354.0,NaN


Уберите лишний символ, преобразуйте признак time к datetime. После этого найдите наибольшую дату и выведите ее без времени.

Подсказка: можно применить метод max() к получившемуся столбцу со временем.

Не забудьте избавиться от пропусков.

Запишите ответ в формате "YYYY-MM-DD".

In [222]:
log.time = log.time.apply(lambda s: str(s)[1:] if str(s).startswith('[') else s)
log.time = pd.to_datetime(log.time)
log.time.max()

Timestamp('2019-04-20 18:10:07')

Используйте оригинальные данные log.csv, столбец time.

Подсказка: можно использовать value_counts().

Найдите минуту, которая встречалась в данных чаще всего. Введите ответ в поле ниже.

In [223]:
log.time.dt.minute.value_counts(ascending=False).head()

36.0    25
31.0    23
6.0     22
50.0    22
14.0    21
Name: time, dtype: int64

Используйте оригинальные данные log.csv, столбец time.

Подсказка: можно использовать value_counts().

Найдите месяц, который встречался в данных реже всего. Введите ответ в поле ниже.

In [224]:
log.time.dt.month.value_counts(ascending=True).head()

4.0    170
2.0    259
3.0    264
1.0    292
Name: time, dtype: int64

In [225]:
log.time[log.time.dt.dayofweek >= 5]

39    2019-01-05 01:54:07
40    2019-01-05 10:44:56
41    2019-01-05 13:13:21
42    2019-01-05 14:36:15
43    2019-01-05 18:09:15
              ...        
991   2019-04-20 12:55:41
992   2019-04-20 14:59:36
993   2019-04-20 17:09:56
994   2019-04-20 18:10:07
995   2019-04-20 09:59:58
Name: time, Length: 283, dtype: datetime64[ns]

Используйте оригинальные данные log.csv, столбец time.

Подсказка: можно использовать value_counts(). Кроме этого, потребуется написать функцию, которая преобразует дату во время дня.

Договоримся, что с 0 до 5 часов - ночь, с 6 до 11 - утро, с 12 до 17 - день, с 18 до 23 - вечер.

Важно: для выполнения задания вам нужно будет избавиться от пропусков только в столбце time. Вспомните, как избавиться от пропусков только по конкретному признаку.

Посчитайте, какое время дня встречается в данных реже всего. Введите ответ в поле ниже: ночь, утро, день или вечер.

In [226]:
def to_day_time(hour):
    if hour >= 0 and hour <= 5:
        return 'ночь'
    if hour >= 6 and hour <= 11:
        return 'утро'        
    if hour >= 12 and hour <= 17:
        return 'день'           
    if hour >= 18 and hour <= 23:
        return 'вечер'                     

day_time = log.time.apply(lambda t: to_day_time(t.hour))
day_time.value_counts(ascending=True)

вечер    227
день     240
утро     253
ночь     265
Name: time, dtype: int64

Давайте повторим то, что мы прошли в этой секции. Напишите код, который создаст признак hour из признака time в датасете log.csv. Для этого:

1. загрузите датасет log.csv в переменную log, дальше работать будем с ней;
2. установите имена столбцов: ['user_id', 'time', 'bet', 'win'];
3. избавьтесь от пропусков в log;
4. приведите переменную time к подходящему для извлечения признаков виду;
5. получите значение часа для каждой строки в переменной time и запишите в столбец hour в log.

Результатом будет таблица log со столбцом hour внутри.

In [227]:
log = pd.read_csv("data/log.csv", header=None, names=['user_id','time', 'bet','win'])
log.dropna(axis=0, inplace=True)
log.time = log.time.apply(lambda s: str(s)[1:] if str(s).startswith('[') else s)
log.time = pd.to_datetime(log.time)
log['hour'] = log.time.apply(lambda t: t.hour)
display(log)


,user_id,time,bet,win,hour
14,Запись пользователя № - user_917,2019-01-02 08:57:36,145732.0,1987653.0,8
29,Запись пользователя № - user_942,2019-01-04 13:59:42,1678321.0,9876543.0,13
151,Запись пользователя № - user_982,2019-01-16 21:54:22,100.0,4749.0,21
189,Запись пользователя № - user_964,2019-01-21 18:34:44,200.0,4667.0,18
205,Запись пользователя № - user_931,2019-01-22 05:26:59,300.0,4319.0,5
...,...,...,...,...,...
967,Запись пользователя № - user_975,2019-04-19 22:25:15,1000.0,6108.0,22
971,Запись пользователя № - user_912,2019-04-20 10:35:49,10554.0,31799.0,10
972,Запись пользователя № - user_926,2019-04-20 10:35:50,10354.0,30244.0,10
976,Запись пользователя № - user_970,2019-04-20 10:35:54,10354.0,30691.0,10


## Заполнение константой

In [228]:
log = pd.read_csv("data/log.csv", header=None, names=['user_id','time', 'bet','win'])
log.time = log.time.apply(lambda s: str(s)[1:] if str(s).startswith('[') else s)
log.time = pd.to_datetime(log.time)
log['hour'] = log.time.apply(lambda t: t.hour)

Используйте оригинальный датасет log.csv. Результат запишите числом в поле ниже.

Подсказка: можно использовать value_counts().

Посчитайте, сколько раз люди приходили, но не делали ставок. Для этого заполните пропуски в столбце bet значением 0 и посчитайте количество таких значений.

In [229]:
log['bet'] = log.bet.fillna(0)
log.bet.value_counts().head()

0.0      515
500.0     50
100.0     48
300.0     42
800.0     42
Name: bet, dtype: int64

## Заполнение с помощью функции

Используйте оригинальный датасет log.csv. Проделайте с ним действия из задания 1, расположенного выше на этой странице, и из пункта "Заполнение с помощью функции".

Результат запишите числом в поле ниже.

Подсказка: можно использовать sum().

Посчитайте, сколько раз участники ставок проиграли деньги. То есть посчитайте количество строк, для которых в столбце win находится

In [230]:
def fill_na_win(row):
    if not pd.isna(row.win):
        return row.win
    if pd.isna(row.bet):
        return 0
    else:
        return -row.bet

log.win = log.apply(lambda row: fill_na_win(row), axis=1)  
log.win.loc[wins < 0]


5     -156789.0
19     -98753.0
67      -5000.0
81    -123981.0
92       -100.0
         ...   
994    -10354.0
995     -9754.0
997    -10454.0
998     -1000.0
999    -10754.0
Name: win, Length: 347, dtype: float64

Создайте признак net, хранящий сумму выигрыша с учетом ставки. Для этого из признака win поэлементно вычтите признак bet и запишите в новый столбец. После этого посчитайте, у скольких людей выигрыш положительный.

In [231]:
log['net'] = log.apply(lambda row: row.win - row.bet, axis=1)
winners = log.loc[log.net > 0]
winners.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 14 to 996
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   user_id  138 non-null    object        
 1   time     133 non-null    datetime64[ns]
 2   bet      138 non-null    float64       
 3   win      138 non-null    float64       
 4   hour     133 non-null    float64       
 5   net      138 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 7.5+ KB


Используйте датасет log.csv, получившийся в результате выполнения предыдущего задания. Посчитайте среднее значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

In [232]:
round(winners.net.mean())

80253

Используйте датасет log.csv, получившийся в результате выполнения первого задания этого блока. Посчитайте медианное значение выигрыша (из столбца net) в тех случаях, когда выигрыш больше 0. Результат округлите до целого, отбросив дробную часть.

In [233]:
round(winners.net.median())

5347

## Что осталось сделать
Используйте модифицированный исходный датасет log.csv.

При модификации датасета log.csv, пропущенные значения в столбцах bet и win замените на 0, cоздайте столбец net, хранящий сумму выигрыша с учетом ставки (для этого из столбца win поэлементно вычтите столбец bet и запишите в новый столбец).

In [234]:
log = pd.read_csv("data/log.csv", header=None, names=['user_id','time', 'bet','win'])
log.time = log.time.apply(lambda s: str(s)[1:] if str(s).startswith('[') else s)
log.time = pd.to_datetime(log.time)
log['hour'] = log.time.apply(lambda t: t.hour)
log['bet'] = log.bet.fillna(0)
log['win'] = log.win.fillna(0)
log['net'] = log.apply(lambda row: row.win - row.bet, axis=1)
log

,user_id,time,bet,win,hour,net
0,Запись пользователя № - user_919,2019-01-01 14:06:51,0.0,0.0,14.0,0.0
1,Запись пользователя № - user_973,2019-01-01 14:51:16,0.0,0.0,14.0,0.0
2,Запись пользователя № - user_903,2019-01-01 16:31:16,0.0,0.0,16.0,0.0
3,Запись пользователя № - user_954,2019-01-01 17:17:51,0.0,0.0,17.0,0.0
4,Запись пользователя № - user_954,2019-01-01 21:31:18,0.0,0.0,21.0,0.0
...,...,...,...,...,...,...
995,Запись пользователя № - user_984,2019-04-20 09:59:58,9754.0,0.0,9.0,-9754.0
996,#error,NaT,10054.0,29265.0,NaN,19211.0
997,#error,NaT,10454.0,0.0,NaN,-10454.0
998,#error,NaT,1000.0,0.0,NaN,-1000.0


Посчитайте, какой процент посещений букмекерской конторы оборачивался ставкой. Для этого поделите количество ставок (значений больше 0) на общее количество посещений конторы. Результат округлите до одного знака после запятой.

In [235]:
len(log.loc[log.bet > 0]) / len(log)

0.485

Посчитайте среднее значение ставки (из столбца bet) в тех случаях, когда ставка была сделана. Результат округлите до целого, отбросив дробную часть.

In [236]:
log.loc[log.bet > 0].bet.mean()

6785.738144329897

Посчитайте средний выигрыш (из столбца net) в тех случаях, когда ставка была сделана. Результат округлите до целого, отбросив дробную часть.

Пояснение: выигрыш в данном случае означает изменение количества денег и может быть отрицательным. В таком случае это проигрыш.

In [237]:
log.loc[log.bet > 0].net.mean()

20421.892783505155

Посчитайте среднюю сумму потерь при проигрыше (из столбца net). Результат округлите до целого, отбросив дробную часть.

Пояснение: ответ должен быть дан в виде отрицательного числа.

In [238]:
log.loc[log.net < 0].net.mean()

-3372.743515850144

Посчитайте, какой процент ставок заканчивается выигрышем, а какой - проигрышем. Сравните эти значения и ответьте, какое из них больше.

In [239]:
bets = log.loc[log.bet > 0]
bets_number = len(bets)

print('Доля выигрышей: {}'.format(len(bets.loc[bets.net > 0]) / bets_number))
print('Доля проигрышей: {}'.format(len(bets.loc[bets.net < 0]) / bets_number))


Доля выигрышей: 0.2845360824742268
Доля проигрышей: 0.7154639175257732


Давайте повторим то, что мы прошли в этой секции. Напишите код, который узнает, чему была равна минимальная ставка и сколько людей сделали такую ставку. Для этого:

1. загрузите датасет log.csv;
2. посчитайте, чему равна минимальная ставка;
3. посчитайте, сколько раз была сделана минимальная ставка, и запишите результат в переменную min_bet_amount в виде целого числа.

In [240]:
log_df = pd.read_csv("data/log.csv", header=None, names=['user_id','time', 'bet','win'])
log_df.time = log_df.time.apply(lambda s: str(s)[1:] if str(s).startswith('[') else s)
min_bet_amount = log_df.bet.value_counts().sort_index().iloc[0]
print(min_bet_amount)

48


## Повторение merge/groupby
Повторим часть предобработки, которую мы должны были выполнить ранее:

In [241]:
# Приведем признак user_id к одному формату в обоих датасетах  
us.user_id = us.user_id.apply(lambda x: x.lower())  
# Избавимся от ошибок в user_id  
log = log[log.user_id != '#error']  
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])  

Это будет нужно для того, чтобы объединить оба датасета и работать с едиными данными для проведения продвинутого анализа.

## Merge

Теперь объединим данные с помощью метода pd.merge():

`pd.merge(dataframe1, dataframe2, on='feature_name')`

Первые два аргумента - таблицы, которые нужно будет объединить.

Третий аргумент - название признака, по которому будем объединять данные. Мы уже привели данные к одинаковому виду, и теперь их можно объединить по признаку 'user_id', чтобы получить полную информацию о пользователе. 

In [242]:
merged = pd.merge(log, us, on='user_id')
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 975 entries, 0 to 974
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  975 non-null    object 
 1   time     975 non-null    object 
 2   bet      470 non-null    float64
 3   win      132 non-null    float64
 4   email    967 non-null    object 
 5   geo      950 non-null    object 
dtypes: float64(2), object(4)
memory usage: 53.3+ KB


## Groupby

Теперь повторим groupby.

Данный метод позволяет сгруппировать данные и применить к ним методы агрегации:

`df.groupby('user_id').win.median().median()`

В данном случае мы группируем данные по признаку user_id.

После этого мы в каждой группе выбираем признак win.

Затем мы берем медиану каждой группы по признаку win и на выходе получаем таблицу, в которой индексом является признак user_id. В этой таблице единственный столбец - медиана по каждой группе (то есть по каждому пользователю).

Наконец, последний вызов median() дает нам медиану по предыдущему столбцу, то есть возвращает одно число.

## Анализ по пользователям
Мы собрали все, что нам нужно, чтобы извлечь какие-то инсайты из данных о пользователях.

Используйте датасет, который получился в результате всех преобразований выше (в том числе, заполнение пропусков). Ответ запишите в поле ниже в виде целого числа (отбросьте дробную часть).

Посчитайте медиану баланса по каждому пользователю. Для этого сгруппируйте по пользователям, возьмите признак net, просуммируйте по каждому пользователю и получите медиану.

In [247]:
merged.groupby('user_id').net.median().median()

,user_id,time,bet,win,email,geo
0,user_919,2019-01-01 14:06:51,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,NaN,NaN,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 4:23:36,300.0,NaN,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...
970,user_932,2019-02-24 22:40:06,NaN,NaN,BraceWalker@bk.ru,Красноярск
971,user_932,2019-03-15 10:56:14,NaN,NaN,BraceWalker@bk.ru,Красноярск
972,user_932,2019-03-18 10:13:24,NaN,NaN,BraceWalker@bk.ru,Красноярск
973,user_932,2019-03-27 12:18:24,NaN,NaN,BraceWalker@bk.ru,Красноярск
